# June 2020

In [1]:
Sequence data from Ensembl (https://useast.ensembl.org/info/data/ftp/index.html).

Filename = Homo_sapiens.GRCh38.ncrna.fa.gz
Num sequences = 58028
Shortest sequence: ENSG00000056678.11 ENST00000411692.1 lncRNA CGGCC
Top 5 longest sequence lengths: {37027, 37852, 49287, 91667, 205012}

SyntaxError: invalid syntax (<ipython-input-1-e5c1993847e6>, line 1)

In [48]:
import pandas as pd
infile='lncRNA.HeLa.features.csv'
scored_sequence = pd.read_csv(infile,header=0)
scored_sequence.head()

,label,seqname,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,-2.037350,ENSG00000259891.1,71,8,9,16,16,5,0,1,...,2,9,20,11,8,8,11,3,19,11
1,-0.484711,ENSG00000215808.4,62,38,44,47,32,30,2,28,...,2,54,31,25,24,42,44,50,56,117
2,-1.929540,ENSG00000236088.10,54,30,37,42,37,15,4,17,...,4,75,33,27,24,46,70,75,52,152
3,-2.027480,ENSG00000272202.1,7,4,7,8,2,2,2,3,...,1,1,1,1,2,0,0,3,0,0
4,-4.424920,ENSG00000180422.3,2,2,2,4,5,2,1,1,...,3,10,4,8,8,4,2,10,9,9


In [49]:
scored_sequence.shape

(1152, 258)

In [50]:
def split_train_test(dataset,train_portion):    
    # Use pandas sample() to randomize the order (i.e. random sample without replacement).
    # Side note. Data frame can be randomized while keeping header in place.
    # This is repeatable with 42.
    middle = int(len(dataset)*train_portion)
    random = dataset.sample(frac=1,random_state=42)
    train_set = random[:middle]
    test_set = random[middle:]
    return train_set,test_set
TRAIN_PORTION=0.8
raw_train, raw_test = split_train_test(scored_sequence,TRAIN_PORTION)
shuffle_train = raw_train.sample(frac=1,random_state=17)
y_train = shuffle_train['label']
X_train_ID = shuffle_train['seqname'] # Save for later?
X_train = shuffle_train.drop(['label','seqname'],axis=1)
X_train.shape
y_train.shape

(921,)

In [51]:
# In first sequence, minimum K-mer count is 312!
X_train[0:1].min(axis=1)
X_train_ID[263]

'ENSG00000235077.1'

In [52]:
# Normalize by row sum. 
X_norm=X_train.div(X_train.sum(axis=1), axis=0)
# After normalization, each row should sum to 1.
X_norm.sum(axis=1)

263    1.0
571    1.0
250    1.0
462    1.0
887    1.0
      ... 
572    1.0
259    1.0
654    1.0
243    1.0
74     1.0
Length: 921, dtype: float64

In [53]:
X_norm[0:1].min(axis=1)

263    0.0
dtype: float64

In [54]:
y_train.shape, X_norm.shape

((921,), (921, 256))

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_norm,y_train)
y_pred=lin.predict(X_norm)
lin.score(X_norm,y_train) # test set == train set i.e. best case scenario

0.40011858145020307

In [62]:
lin.fit(X_norm[800:],y_train[800:])
y_pred=lin.predict(X_norm)
lin.score(X_norm[:800],y_train[:800]) # train/test = 800/121

-2.5668628618205434

In [56]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_train,y_pred)
mse  # test set == train set i.e. best case scenario

2.0005105686424307

In [63]:
scores=cross_val_score(lin,X_norm,y_train,cv=3)
scores

array([-0.71572244, -0.63657974, -0.53192183])